# Migrating CSV Data into a Relational Database

# Step 1: Import CSV Data into Pandas Dataframes

In [1]:
import pandas as pd

#### Clean_Hawaii_Measurments

In [2]:
measurments_df = pd.read_csv('../resources/clean_hawaii_measurements.csv')

In [3]:
measurments_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.00,73


In [4]:
measurments_df.count()

station    19550
date       19550
prcp       19550
tobs       19550
dtype: int64

In [5]:
measurments_df.dtypes

station     object
date        object
prcp       float64
tobs         int64
dtype: object

#### Hawaii_Stations

In [6]:
stations_df = pd.read_csv('../resources/hawaii_stations.csv')

In [7]:
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [8]:
stations_df.count()

station      9
name         9
latitude     9
longitude    9
elevation    9
dtype: int64

In [9]:
stations_df.dtypes

station       object
name          object
latitude     float64
longitude    float64
elevation    float64
dtype: object

# Step 2: Mapping Data and Connecting the Schema

In [10]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, Float, Text

In [11]:
engine = create_engine('sqlite:///../resources/database/hawaii.sqlite')

In [12]:
Base = declarative_base(bind=engine)

In [13]:
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    date = Column(Text)
    prcp = Column(Float(6,2))
    tobs = Column(Integer)
    
    
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    name = Column(String(75))
    latitude = Column(Float(6,4))
    longitude = Column(Float(6,4))
    elevation = Column(Float(6,2))

In [14]:
Base.metadata.create_all()

# Step 3: Migrating Dataframe into Schema

In [15]:
data_measurements = measurments_df.to_dict(orient='records')
data_stations = stations_df.to_dict(orient='records')

In [16]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [17]:
table_measurement = sqlalchemy.Table('measurement', metadata, autoload=True)
table_station = sqlalchemy.Table('station', metadata, autoload=True)

In [18]:
engine.execute(table_measurement.delete())
engine.execute(table_station.delete())

In [19]:
engine.execute(table_measurement.insert(), data_measurements)
engine.execute(table_station.insert(), data_stations)

### Testing the tables that store the migrated data

In [20]:
engine.execute("SELECT * FROM measurement LIMIT 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.0, 73)]

In [21]:
engine.execute("SELECT * FROM station LIMIT 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]